# Ansatz Comparisons for the Transverse Ising Model

This notebook is supplementary to the Ising Model notebook.

The goal here is two showcase two different ansatz for the TIM, and discuss why one may work better than the other. We will compare it against the classical solution (which can be found in the notebook titled "Classical Ising Solution"). 

## Initial Ansatz

Recall from our Ising Notebook, for the two site Ising Model, we used the following ansatz:

$$ | \psi (\theta_1, \theta_2)\rangle = U(\theta_1)|0\rangle \otimes U(\theta_2)|0\rangle $$

For our first ansatz, we will simply extend this ansatz to four sites. That is:

$$ | \psi(\theta_1, \theta_2, \theta_3, \theta_4) \rangle = \bigotimes_{i=1}^4 U(\theta_i)| 0 \rangle $$

We will now see how this works. Note that we are considering **periodic boundary conditions**, meaning that the fourth site can interact with the first site. We can think of this 4 site Hamiltonian as a square with a spin on each corner. 



In [2]:
import numpy as np
import math as m
import matplotlib.pyplot as plt
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, execute, BasicAer, IBMQ
from qiskit.visualization import plot_histogram, plot_bloch_multivector
#from qiskit.extensions import Initialize
#from qiskit_textbook.tools import random_state, array_to_latex # This import isn't working for me

import qiskit
print(qiskit.__qiskit_version__) # I add this so we know what versions are being used

{'qiskit-terra': '0.14.1', 'qiskit-aer': '0.5.2', 'qiskit-ignis': '0.3.0', 'qiskit-ibmq-provider': '0.7.2', 'qiskit-aqua': '0.7.1', 'qiskit': '0.19.3'}


In [48]:
def ansatz(params, qc): 
    # params should be a numpy array [theta1, theta2, theta3, theta4]
    # This is applied in expectation, and it is assumed to be applied on a |0000> system
    # Sets theta = param, and the next two to 0. Last argument is the qubit number
    
    qc.u3(params[0], 0, 0, 0) 
    qc.u3(params[1], 0, 0, 1)
    qc.u3(params[2], 0, 0, 2) 
    qc.u3(params[3], 0, 0, 3) 
    print(qc.draw())
    return qc

def entangled_ansatz(params, qc):
    # params should be five angles
    
    qc.u3(params[0],0,0,0)
    qc.cx(0,1)
    qc.cx(0,2)
    qc.cx(0,3)
    qc.u3(params[1], 0, 0, 0) 
    qc.u3(params[2], 0, 0, 1)
    qc.u3(params[3], 0, 0, 2) 
    qc.u3(params[4], 0, 0, 3)
    print(qc.draw())
    return qc

def expZZ(index, qcO, shots_number = 1024):
    # for a given index, we measure index , index+1
    qc = qcO.copy()
    qc.measure(index, 0)
    qc.measure((index + 1) % 4, 1) # mod 4 to allow for periodic boundary conditions
    backend = BasicAer.get_backend('qasm_simulator')
    counts = execute(qc, backend, shots=shots_number).result().get_counts() # executs the job
    # if 11, 00, return 1
    # if 01, 10, return -1    
    freq_pos = 0.0
    freq_neg = 0.0
    
    for key in counts.keys():
        if (key == '00') or (key == '11'):
            #print(key)
            #print('+')
            freq_pos = freq_pos + counts[key]
        else: #01, 10
            #print(key)
            #print('-')
            freq_neg = freq_neg + counts[key]
    #normalize
    freq_pos = freq_pos / shots_number
    freq_neg = freq_neg / shots_number
    #print(freq_pos)
    #print(freq_neg)
    #print(freq_pos + freq_neg)
    #print(counts)
    expZZ = (freq_pos - freq_neg)
    return expZZ



def expX(index, qcO, shots_number = 1024): 
    # for a given index, we measure index
    qc = qcO.copy()
    qc.h(index)
    qc.measure(index, 0) # we always measure to the first, so our results are 00 and 01
    
    backend = BasicAer.get_backend('qasm_simulator')
    counts = execute(qc, backend, shots=shots_number).result().get_counts() # executes the job
    freq_pos = 0.0
    freq_neg = 0.0
    
    for key in counts.keys():
        if key == '00': #the result is printed in reverse order
            #print(key)
            freq_pos = freq_pos + counts[key]
            
        else:
            freq_neg = freq_neg + counts[key]
            
    freq_pos = freq_pos / shots_number
    freq_neg = freq_neg / shots_number
    expX = freq_pos - freq_neg
    #print("expX is " + str(expX))
    #print("======")
    
    return expX



def expectation(params, h, shots_number = 1024): 
    # first, we run the quantum circuit
    # Then, using the resulting measurement, we use the numbers to calculate the expectation value
    # This function is hard-coded for a specific H. (I may change this later)
    # Since H = sigma^z, let's assume that we are in the z-basis. Then, we need only find the amplitudes of being 
    # in |0> and |1>. 
    # <H> = (Counts(0) - Counts(1)) / Normalization
    
    # Creating the circuit
    q = QuantumRegister(4)
    c = ClassicalRegister(2) # we only need two classical bits since we at most take 2 measurements
    qc = QuantumCircuit(q, c)
    
    ansatz(params, qc) # Applying the ansatz preparation on a given |0>
    
    expX_tot = 0.0
    expZZ_tot = 0.0
    i = 0
    while i < 4: #iterates 01, 12, 23, 30 for the ZZ term
        expX_tot = expX_tot + expX(i, qc)
        expZZ_tot = expZZ_tot + expZZ(i, qc)
        i += 1
    X_contribution = -h * expX_tot
    ZZ_contribution = -1*expZZ_tot
    print("The X contribution is: " + str(X_contribution))
    print("The ZZ contribution is: " + str(ZZ_contribution))
    exp =  X_contribution + ZZ_contribution
    return exp


In [49]:
theta1 = 0
theta2 = 0
theta3 = 0
theta4 = 0

params = np.array([theta1, theta2, theta3, theta4])
h = 0.1

q = QuantumRegister(4)
c = ClassicalRegister(2) # we only need two classical bits since we at most take 2 measurements
qc = QuantumCircuit(q, c)

expectation(params, h)


entangled_ansatz(np.array([0,0,0,0,0]), qc)

       ┌───────────┐
q73_0: ┤ U3(0,0,0) ├
       ├───────────┤
q73_1: ┤ U3(0,0,0) ├
       ├───────────┤
q73_2: ┤ U3(0,0,0) ├
       ├───────────┤
q73_3: ┤ U3(0,0,0) ├
       └───────────┘
c73_0: ═════════════
                    
c73_1: ═════════════
                    
The X contribution is: -0.012109375
The ZZ contribution is: -4.0
       ┌───────────┐                            ┌───────────┐
q72_0: ┤ U3(0,0,0) ├──■────■─────────────────■──┤ U3(0,0,0) ├
       └───────────┘┌─┴─┐  │  ┌───────────┐  │  └───────────┘
q72_1: ─────────────┤ X ├──┼──┤ U3(0,0,0) ├──┼───────────────
                    └───┘┌─┴─┐├───────────┤  │               
q72_2: ──────────────────┤ X ├┤ U3(0,0,0) ├──┼───────────────
                         └───┘└───────────┘┌─┴─┐┌───────────┐
q72_3: ────────────────────────────────────┤ X ├┤ U3(0,0,0) ├
                                           └───┘└───────────┘
c72_0: ══════════════════════════════════════════════════════
                                          

## References

https://arxiv.org/pdf/1804.03719.pdf